In [1]:
import threading
import multiprocessing
import numpy as np
import tensorflow as tf
from AtariEnvironment import *
from ACERNetwork import * 
from ReplayBuffer import *
from Worker import * 
from Trainer import train,train_acer, test
import gym as gym
from time import sleep
from time import time
from utils import preprocess

network_t = "cnn"
GAME_NAME="Breakout-v0"
#GAME_NAME = "LunarLander-v2"
TB_DIR = "saves/breakout/3em4_8online/"
env=Atari_Environment(gym.make(GAME_NAME))
#env = gym.make(GAME_NAME)
state= env.reset()
training_finished = False
LEARNING_RATE = 2e-4
print(env.action_space)

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


Discrete(4)


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [2]:
from tensorflow import Session, ConfigProto
import os
os.environ["CUDA_VISIBLE_DEVICES"]=""
config = ConfigProto()
config.gpu_options.allow_growth=True
Session(config=config).close()

In [3]:
session = tf.Session()
tf.reset_default_graph()

with tf.Session() as sess:
    with tf.device('/cpu:0'):
        G_Agent=ACERNetwork(sess,env,"global",network_type=network_t)
        agent = ACERNetwork(sess,env,str(0),G_Agent, network_type=network_t)
        sess.run(tf.global_variables_initializer())
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))
        agent.update_target()
        print(agent.get_pi(state))
        print(G_Agent.get_pi(state))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


[[0.2384395  0.23251908 0.26794815 0.26109326]]
[[0.24325493 0.27135077 0.23667349 0.2487208 ]]
[[0.24325493 0.27135077 0.23667349 0.2487208 ]]
[[0.24325493 0.27135077 0.23667349 0.2487208 ]]


In [4]:
def start_method(game_name=GAME_NAME, num_agents = 16, restore = False, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    workers=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       

    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            r_memory = ReplayBuffer(50000)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=5)
            

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
            # Create a process for each worker
            for thread_id in range(num_agents):
                if thread_id == 0:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,"eval", r_memory, 20, 0.99, 1000000, 2))
                    processes.append(threading.Thread(target=workers[thread_id].work_and_eval_acer, args=(saver,TB_DIR)))
                else:
                    workers.append(Worker(agents[thread_id],envs[thread_id],sess,str(thread_id), r_memory, 20, 0.99, 1000000, 2))
                    processes.append(threading.Thread(target=workers[thread_id].work_acer))
            # Start all the processes
            for p in processes:
                p.daemon = True
                p.start()
                
            # Join the processes, so we get this thread back.
            coord.join(processes)


In [5]:
tf.reset_default_graph()
start_method()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1  using  2  per online step
worker_2  using  2  per online step
worker_3  using  2  per online step
worker_4  using  2  per online step
worker_5  using  2  per online step
worker_6  using  2  per online step
worker_7  using  2  per online step
worker_8  using  2  per online step
worker_9  using  2  per online step
worker_10  using  2  per online step
worker_11  using  2  per online step
worker_12  using  2  per online step
worker_13  using  2  per online step
worker_14 worker_15  using  2  per online step
 using  2  per online step


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  183  Running/Max:  1.0 1.0  Frames: 183
pi: [[0.2420402  0.25036615 0.25049993 0.25709376]]
Saving Model
Worker  worker_eval At  390  Running/Max:  1.0 1.0  Frames: 390
pi: [[0.24226117 0.25008187 0.2500632  0.25759375]]
Saving Model
Worker  worker_eval At  595  Running/Max:  1.0 1.0  Frames: 596
pi: [[0.24287936 0.25160915 0.2477896  0.25772187]]
Saving Model
Worker  worker_eval At  908  Running/Max:  1.2000000000000002 3.0  Frames: 909
pi: [[0.23296131 0.24994332 0.2578998  0.2591956 ]]
Saving Model
Worker  worker_eval At  1237  Running/Max:  1.3800000000000003 3.0  Frames: 1237
pi: [[0.18973176 0.3145706  0.2693466  0.226351  ]]
Saving Model
Worker  worker_eval At  1505  Running/Max:  1.4420000000000004 3.0  Frames: 1505
pi: [[0.24723049 0.249651   0.23407112 0.26904732]]
Saving Model
Worker  worker_eval At  1704  Running/Max:  1.3978000000000004 3.0  Frames: 1704
pi: [[0.24054039 0.24906193 0.25603315 0.25436452]]
Saving Model
Worker  worker_eval At  1961  R

Worker  worker_eval At  14912  Running/Max:  2.5368136627843723 6.0  Frames: 14912
pi: [[0.23901924 0.25319156 0.30682805 0.2009611 ]]
Saving Model
Worker  worker_eval At  15172  Running/Max:  2.4831322965059353 6.0  Frames: 15172
pi: [[0.21967624 0.2511846  0.38484627 0.144293  ]]
Saving Model
Worker  worker_eval At  15408  Running/Max:  2.4348190668553418 6.0  Frames: 15408
pi: [[0.23706976 0.23792814 0.3312161  0.19378605]]
Saving Model
Worker  worker_eval At  15584  Running/Max:  2.1913371601698075 6.0  Frames: 15584
pi: [[0.25456968 0.25338793 0.3395021  0.15254027]]
Saving Model
Worker  worker_eval At  15840  Running/Max:  2.172203444152827 6.0  Frames: 15840
pi: [[0.24737146 0.26067564 0.31729427 0.17465866]]
Saving Model
Worker  worker_eval At  16090  Running/Max:  2.1549830997375445 6.0  Frames: 16090
pi: [[0.24089359 0.2577475  0.32920873 0.17215009]]
Saving Model
Worker  worker_eval At  16337  Running/Max:  2.1394847897637903 6.0  Frames: 16337
pi: [[0.25743818 0.27339813 0.

...

Worker  worker_eval At  1789567  Running/Max:  28.2880754025205 82.0  Frames: 607641
pi: [[9.8588437e-01 1.2871299e-02 1.1088015e-03 1.3560655e-04]]
Saving Model
Worker  worker_eval At  1792278  Running/Max:  27.45926786226845 82.0  Frames: 608545
pi: [[4.3773203e-04 2.4994220e-06 1.2672015e-11 9.9955970e-01]]
Saving Model
Worker  worker_eval At  1795073  Running/Max:  26.913341076041604 82.0  Frames: 609477
pi: [[6.5018569e-33 1.0000000e+00 3.6953958e-36 0.0000000e+00]]
Saving Model
Worker  worker_eval At  1800254  Running/Max:  29.022006968437445 82.0  Frames: 611204
pi: [[1.7776500e-04 9.7747600e-01 2.0035816e-06 2.2344241e-02]]
Saving Model
Worker  worker_eval At  1804580  Running/Max:  29.5198062715937 82.0  Frames: 612646
pi: [[7.0759279e-07 9.9999893e-01 3.4846528e-07 2.4674010e-10]]
Saving Model
Worker  worker_eval At  1808306  Running/Max:  29.067825644434333 82.0  Frames: 613888
pi: [[2.6066243e-03 9.9689025e-01 1.7737657e-04 3.2570329e-04]]
Saving Model
Worker  worker_eval A

Worker  worker_eval At  2000118  Running/Max:  32.99843813383137 82.0  Frames: 677830
pi: [[2.9257705e-13 1.0000000e+00 1.1609074e-12 9.7014705e-12]]
Saving Model
Worker  worker_eval At  2004872  Running/Max:  33.598594320448235 82.0  Frames: 679412
pi: [[3.11338226e-08 9.68038023e-01 3.19481939e-02 1.36861645e-05]]
Saving Model
Worker  worker_eval At  2008160  Running/Max:  32.738734888403414 82.0  Frames: 680508
pi: [[4.1812904e-09 9.9998713e-01 4.5977795e-09 1.2886759e-05]]
Saving Model
Worker  worker_eval At  2013347  Running/Max:  33.66486139956307 82.0  Frames: 682237
pi: [[4.6156137e-09 9.9999988e-01 2.3283606e-09 1.2235117e-07]]
Saving Model
Worker  worker_eval At  2018148  Running/Max:  34.398375259606766 82.0  Frames: 683837
pi: [[9.4292744e-07 9.9978513e-01 6.6568312e-08 2.1384083e-04]]
Saving Model
Worker  worker_eval At  2022313  Running/Max:  34.35853773364609 82.0  Frames: 685227
pi: [[9.8355699e-08 9.9999869e-01 5.4183414e-07 6.4151141e-07]]
Saving Model
Worker  worker_

Worker  worker_eval At  2229397  Running/Max:  37.11484115434047 82.0  Frames: 754253
pi: [[3.89483422e-02 8.91492665e-01 6.95589855e-02 1.22977974e-08]]
Saving Model
Worker  worker_eval At  2234910  Running/Max:  38.50335703890642 82.0  Frames: 756091
pi: [[2.2915225e-04 1.2559450e-01 8.7417632e-01 1.2475067e-10]]
Saving Model
Worker  worker_eval At  2239409  Running/Max:  38.75302133501578 82.0  Frames: 757590
pi: [[6.2119255e-12 1.0000000e+00 3.8029309e-14 7.7532897e-14]]
Saving Model
Worker  worker_eval At  2244079  Running/Max:  38.577719201514206 82.0  Frames: 759149
pi: [[7.7186942e-06 9.9064320e-01 9.3489559e-03 1.8579986e-07]]
Saving Model
Worker  worker_eval At  2248532  Running/Max:  38.11994728136278 82.0  Frames: 760633
pi: [[1.1852893e-11 1.0000000e+00 4.5714843e-12 3.6798910e-15]]
Saving Model
Worker  worker_eval At  2251761  Running/Max:  36.80795255322651 82.0  Frames: 761708
pi: [[0.03622217 0.5643279  0.29709756 0.10235243]]
Saving Model
Worker  worker_eval At  22562

Worker  worker_eval At  2479287  Running/Max:  40.765720548978926 82.0  Frames: 837552
pi: [[0.00616626 0.96611017 0.01760093 0.01012258]]
Saving Model
Worker  worker_eval At  2482701  Running/Max:  39.28914849408103 82.0  Frames: 838688
pi: [[6.252215e-14 1.000000e+00 8.817167e-15 3.849339e-13]]
Saving Model
Worker  worker_eval At  2487915  Running/Max:  39.76023364467293 82.0  Frames: 840427
pi: [[0.08714478 0.00374195 0.02357784 0.8855354 ]]
Saving Model
Worker  worker_eval At  2494366  Running/Max:  41.284210280205635 82.0  Frames: 842579
pi: [[1.0861583e-12 1.0000000e+00 2.3107842e-12 7.9396816e-14]]
Saving Model
Worker  worker_eval At  2498938  Running/Max:  41.45578925218507 82.0  Frames: 844102
pi: [[3.6031269e-03 9.9593651e-01 4.1847475e-04 4.1806208e-05]]
Saving Model
Worker  worker_eval At  2503297  Running/Max:  40.71021032696656 82.0  Frames: 845555
pi: [[2.0749651e-06 9.9998498e-01 1.1131422e-05 1.8752237e-06]]
Saving Model
Worker  worker_eval At  2508019  Running/Max:  4

Worker  worker_eval At  2745767  Running/Max:  43.454713983256354 82.0  Frames: 926377
pi: [[5.4348804e-21 1.0000000e+00 5.5335423e-20 4.6054388e-13]]
Saving Model
Worker  worker_eval At  2751082  Running/Max:  44.60924258493072 82.0  Frames: 928149
pi: [[2.2083979e-10 1.1239894e-07 2.2310431e-11 9.9999988e-01]]
Saving Model
Worker  worker_eval At  2756140  Running/Max:  44.24831832643765 82.0  Frames: 929836
pi: [[5.9002709e-14 1.7569432e-08 7.1628664e-15 1.0000000e+00]]
Saving Model
Worker  worker_eval At  2760247  Running/Max:  43.223486493793885 82.0  Frames: 931204
pi: [[3.9870558e-11 9.9999630e-01 3.8163712e-08 3.6433717e-06]]
Saving Model
Worker  worker_eval At  2764495  Running/Max:  42.7011378444145 82.0  Frames: 932621
pi: [[3.0211615e-09 4.8401561e-02 1.0635914e-07 9.5159835e-01]]
Saving Model
Worker  worker_eval At  2768830  Running/Max:  41.731024059973045 82.0  Frames: 934065
pi: [[1.4631518e-14 1.0000000e+00 2.0210802e-08 4.7892557e-09]]
Saving Model
Worker  worker_eval 

In [ ]:
TB_DIR = "saves/breakout/3m4r3_offline/"
LEARNING_RATE = 3e-4
tf.reset_default_graph()
start_method()

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:100: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


worker_1worker_2  using  2  per online stepworker_3  using  2  per online step
worker_4  using  2  per online step
 
 using worker_5  using  2  per online step
 2  per online step
worker_6  using  2  per online step
worker_7  using  2  per online step
worker_8  using  2  per online step
worker_9  using  2  per online step
worker_10  using  2  per online step
worker_11  using  2  per online step
worker_12  using  2  per online step
worker_13  using  2  per online step
worker_14  using  2  per online step
worker_15  using  2  per online step


/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/juull100/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


Worker  worker_eval At  149  Running/Max:  0.9 0.0  Frames: 149
pi: [[0.24748282 0.24646269 0.2530324  0.25302207]]
Saving Model
Worker  worker_eval At  461  Running/Max:  1.11 3.0  Frames: 461
pi: [[0.24918325 0.2455013  0.25312978 0.25218564]]
Saving Model
Worker  worker_eval At  713  Running/Max:  1.199 3.0  Frames: 713
pi: [[0.24224555 0.2455732  0.2564949  0.25568628]]
Saving Model
Worker  worker_eval At  938  Running/Max:  1.1791000000000003 3.0  Frames: 938
pi: [[0.21032755 0.22439766 0.2868728  0.27840197]]
Saving Model
Worker  worker_eval At  1320  Running/Max:  1.4611900000000002 4.0  Frames: 1320
pi: [[0.05902724 0.23251598 0.68983775 0.01861904]]
Saving Model
Worker  worker_eval At  1557  Running/Max:  1.5150710000000003 4.0  Frames: 1557
pi: [[0.15466423 0.59715027 0.1674323  0.08075324]]
Saving Model
Worker  worker_eval At  1770  Running/Max:  1.4635639000000005 4.0  Frames: 1770
pi: [[0.18550341 0.48125446 0.19738771 0.13585442]]
Saving Model
Worker  worker_eval At  1938

Worker  worker_eval At  13481  Running/Max:  1.5520807143628066 4.0  Frames: 13481
pi: [[0.24136753 0.21715087 0.315728   0.22575358]]
Saving Model
Worker  worker_eval At  13731  Running/Max:  1.5968726429265259 4.0  Frames: 13731
pi: [[0.22176656 0.20555632 0.40443772 0.1682394 ]]
Saving Model
Worker  worker_eval At  13983  Running/Max:  1.6371853786338733 4.0  Frames: 13983
pi: [[0.24760498 0.23880589 0.28299746 0.23059167]]
Saving Model
Worker  worker_eval At  14232  Running/Max:  1.673466840770486 4.0  Frames: 14232
pi: [[0.23668131 0.26869068 0.27694932 0.21767865]]
Saving Model
Worker  worker_eval At  14483  Running/Max:  1.7061201566934374 4.0  Frames: 14483
pi: [[0.2807118  0.28221756 0.24766986 0.1894008 ]]
Saving Model
Worker  worker_eval At  14756  Running/Max:  1.7355081410240936 4.0  Frames: 14756
pi: [[0.24933109 0.24643631 0.3863469  0.11788571]]
Saving Model
Worker  worker_eval At  15020  Running/Max:  1.7619573269216842 4.0  Frames: 15020
pi: [[0.25285184 0.2746145  0.

Worker  worker_eval At  51214  Running/Max:  2.3964010659554784 5.0  Frames: 28185
pi: [[0.20156322 0.37065813 0.27333266 0.154446  ]]
Saving Model
Worker  worker_eval At  51670  Running/Max:  2.156760959359931 5.0  Frames: 28335
pi: [[0.19012852 0.34625044 0.31886208 0.14475904]]
Saving Model
Worker  worker_eval At  52162  Running/Max:  1.9410848634239377 5.0  Frames: 28503
pi: [[0.20069957 0.36016434 0.29400656 0.14512953]]
Saving Model
Worker  worker_eval At  53046  Running/Max:  2.046976377081544 5.0  Frames: 28794
pi: [[0.2094979  0.36017752 0.26973987 0.16058467]]
Saving Model
Worker  worker_eval At  53791  Running/Max:  2.0422787393733897 5.0  Frames: 29044
pi: [[0.22209369 0.36342016 0.24052228 0.17396392]]
Saving Model
Worker  worker_eval At  54717  Running/Max:  2.1380508654360506 5.0  Frames: 29352
pi: [[0.22166447 0.3357046  0.27472818 0.16790274]]
Saving Model
Worker  worker_eval At  55600  Running/Max:  2.2242457788924455 5.0  Frames: 29647
pi: [[0.11621019 0.693317   0.0

In [ ]:
def test_method(game_name=GAME_NAME, num_agents = 1, restore = True, save_path=TB_DIR):
        
    envs=[]
    agents=[]
    processes=[]
    
    for _ in range(num_agents):
        #a_env = gym.make(game_name)
        #a_env = Custom_Environment(gym.make(game_name))
        a_env = Atari_Environment(gym.make(game_name), clipreward=True)
        envs.append(a_env)
       


    with tf.Session() as sess:
        with tf.device('/cpu:0'):
            coord = tf.train.Coordinator()
            G_Agent=ACERNetwork(sess,envs[0],"global",network_type=network_t,lr=LEARNING_RATE,decay=0.99)
            for agent_num in range(num_agents):
                agents.append(ACERNetwork(sess,envs[0],str(agent_num),G_Agent,network_type=network_t))
            saver = tf.train.Saver(max_to_keep=2)

            if restore:
                ckpt = tf.train.get_checkpoint_state(save_path+"checkpoints")
                saver.restore(sess,ckpt.model_checkpoint_path)
            else:
                sess.run(tf.global_variables_initializer())
                
            for thread_id in range(num_agents):
                processes.append(threading.Thread(target=test, args=(agents[thread_id],
                envs[thread_id], 2, True)))

            for p in processes:
                p.daemon = True
                p.start()

                
            for p in processes:
                p.join()


In [ ]:
tf.reset_default_graph()
test_method()